# Registry workflows

In [34]:
# set lifemonitor root path
lifemonitor_root = "/mnt/data/projects/@crs4/EOSC-Life/Repositories/life_monitor_rachk8s"
%cd -q {lifemonitor_root}

In [35]:
import requests
from examples import utils # just a simple utility to share code between netbooks

In [36]:
# LifeMonitor URLs
lm_base_url = "https://localhost:8443"

In [37]:
# fetch the token
token = utils.fetch_registry_token('seek')
utils.pp(token)

{ 'access_token': '1CFwLsuycZ9EznhokfSXQtrYFUw3AfHWrVVYsNZEtp',
  'expires_in': 864000,
  'scope': 'read write',
  'token_type': 'Bearer'}


In [38]:
# HTTP settings to connect to LifeMonitor
s = requests.session() # actually not mandatory, but just to share some settings among requests

# if you use self-signed certificates,
# you have to uncomment the line below to disable SSL verification
s.verify = False

# Update headers with the OAuth2 token
s.headers.update({'Authorization': f"Bearer {token['access_token']}"})# HTTP settings to connect to LifeMonitor

In [41]:
# Get workflows
response = s.get(f"{lm_base_url}/workflows")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_workflows = response.json()
utils.pp(registry_workflows)

{'items': []}


In [42]:
# Pick a user registry
response = s.get(f"{lm_base_url}/users")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_users = response.json()
assert len(registry_users)>0 , "Unexpected number of users. We need at least one registered user"
identities = [u for u in registry_users['items'] if u['identity']['sub'] == '2']
assert len(identities) == 1, "Unable to find the user with identity.sub == 2"
utils.pp(user)

{ 'id': 9,
  'identity': { 'provider': { 'name': 'seek',
                              'type': 'seek',
                              'uri': 'https://seek:3000',
                              'userinfo_endpoint': 'https://seek:3000/people/current?format=json'},
                'sub': '2'},
  'username': 't92qXvcogL'}


In [43]:
# let's say that 'user1' is the current_user
# Notice identities
current_user = registry_users['items'][0]
utils.pp(current_user)

{ 'id': 11,
  'identity': { 'provider': { 'name': 'seek',
                              'type': 'seek',
                              'uri': 'https://seek:3000',
                              'userinfo_endpoint': 'https://seek:3000/people/current?format=json'},
                'sub': '2'},
  'username': 'WyFlHUPlGW'}


In [44]:
# get 'user1' info
user1 = utils.get_seek_user_info('user1')
utils.pp(user1)

https://seek:3000/people
{'user1': {'id': '2', 'username': 'user1'}, 'user2': {'id': '3', 'username': 'user2'}}
{ 'attributes': { 'avatar': None,
                  'description': None,
                  'expertise': None,
                  'first_name': 'First',
                  'last_name': 'User',
                  'mbox_sha1sum': 'afbdc5cf6a75764623cd1a4abb114a15e1436f5e',
                  'orcid': None,
                  'project_positions': None,
                  'title': 'First User',
                  'tools': None},
  'id': '2',
  'links': {'self': '/people/2'},
  'meta': { 'api_version': '0.3',
            'base_url': 'https://seek:3000',
            'created': '2020-10-06T13:18:25.738Z',
            'modified': '2020-10-06T13:18:25.738Z',
            'uuid': '56f77130-ea04-0138-8cef-0242c0a8ee02'},
  'relationships': { 'assays': {'data': []},
                     'data_files': {'data': []},
                     'documents': {'data': []},
                     'events': {'da

In [45]:
# Get workflows associated with the user ID '2'
user_workflows = utils.get_seek_user_workflows('user1')
utils.pp(user_workflows)

[ { 'attributes': {'title': 'basefreqsum-invalid'},
    'id': '9',
    'links': {'self': '/workflows/9'},
    'type': 'workflows'},
  { 'attributes': {'title': 'sort-and-change-case-invalid'},
    'id': '7',
    'links': {'self': '/workflows/7'},
    'type': 'workflows'},
  { 'attributes': {'title': 'basefreqsum'},
    'id': '6',
    'links': {'self': '/workflows/6'},
    'type': 'workflows'},
  { 'attributes': {'title': 'sort-and-change-case'},
    'id': '5',
    'links': {'self': '/workflows/5'},
    'type': 'workflows'}]


In [46]:
# Get details of the workflow 'sort-and-change-case' (id '5')
workflow = utils.get_seek_user_workflow('user1', '5')
utils.pp(workflow)

{ 'attributes': { 'content_blobs': [ { 'content_type': 'application/zip',
                                       'link': 'https://seek:3000/workflows/5/content_blobs/5',
                                       'md5sum': '008e3381821327156968574cf2567f22',
                                       'original_filename': 'ro-crate-galaxy-sortchangecase.crate.zip',
                                       'sha1sum': '871152e6fe0908579e89170df75953e420e7b73f',
                                       'size': 7087,
                                       'url': None}],
                  'created_at': '2020-10-06T14:37:07.112Z',
                  'description': 'sort lines and change text to upper case',
                  'discussion_links': None,
                  'internals': { 'inputs': [ { 'description': 'runtime '
                                                              'parameter for '
                                                              'tool Sort',
                                

In [47]:
# Prepare the JSON data required to post the workflow on LifeMonitor
workflow_uuid = workflow['meta']['uuid']
workflow_version = str(workflow["attributes"]["versions"][0]['version']) # pick the first version
workflow_name = workflow["attributes"]["title"]
workflow_roc_link = f'{workflow["attributes"]["content_blobs"][0]["link"]}/download'
post_data = {
    'uuid': workflow_uuid,
    'version': workflow_version,
    'name': workflow_name,
    'roc_link': workflow_roc_link,
    'submitter_id': user1['id']
}
utils.pp(post_data)

{ 'name': 'sort-and-change-case',
  'roc_link': 'https://seek:3000/workflows/5/content_blobs/5/download',
  'submitter_id': '2',
  'uuid': '55203d00-ea0f-0138-9780-0242c0a8ee04',
  'version': '1'}


In [48]:
# trigger the registration of a the workflow 'sort-and-change-case' on LifeMonitor
response = s.post(f"{lm_base_url}/workflows", json=post_data)
assert response.status_code == 201, "Error: status code {} !!!".format(response.status_code)
data = response.json()
print(data)
wf_uuid = data['wf_uuid']
wf_version = data['wf_version']

{'wf_uuid': '55203d00-ea0f-0138-9780-0242c0a8ee04', 'wf_version': '1'}


In [49]:
# Get workflows
response = s.get(f"{lm_base_url}/workflows")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_workflows = response.json()
utils.pp(registry_workflows)

{ 'items': [ { 'name': 'sort-and-change-case',
               'roc_link': 'https://seek:3000/workflows/5/content_blobs/5/download',
               'uuid': '55203d00-ea0f-0138-9780-0242c0a8ee04',
               'version': '1'}]}


In [51]:
# Get workflow details
response = s.get(f"{lm_base_url}/workflows/{workflow_uuid}/{workflow_version}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
workflow = response.json()
utils.pp(workflow)

{ 'name': 'sort-and-change-case',
  'roc_link': 'https://seek:3000/workflows/5/content_blobs/5/download',
  'uuid': '55203d00-ea0f-0138-9780-0242c0a8ee04',
  'version': '1'}


In [52]:
# Get workflow status
response = s.get(f"{lm_base_url}/workflows/{workflow_uuid}/{workflow_version}/status")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
status = response.json()
utils.pp(status)

{ 'aggregate_test_status': 'all_passing',
  'latest_builds': [ { 'build_id': '4',
                       'instance': { 'name': 'test1',
                                     'service': { 'resource': 'job/test/',
                                                  'type': 'jenkins_testing_service',
                                                  'url': 'http://jenkins:8080/',
                                                  'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'},
                                     'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'},
                       'last_logs': 'Started by user Admin Jenkins\n'
                                    'Running as SYSTEM\n'
                                    'Building in workspace '
                                    '/var/jenkins_home/workspace/test\n'
                                    '[test] $ /bin/sh -xe '
                                    '/tmp/jenkins2038142812728915165.sh\n'
                                    '+ ls

In [55]:
# Get workflow status
response = s.get(f"{lm_base_url}/workflows/{wf_uuid}/{wf_version}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
latest_version = response.json()
utils.pp(latest_version)

{ 'name': 'sort-and-change-case',
  'roc_link': 'https://seek:3000/workflows/5/content_blobs/5/download',
  'uuid': '55203d00-ea0f-0138-9780-0242c0a8ee04',
  'version': '1'}


In [56]:
# Get workflow test suites
response = s.get(f"{lm_base_url}/workflows/{workflow_uuid}/{workflow_version}/suites")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
suites = response.json()
utils.pp(suites)

{ 'items': [ { 'instances': [ { 'name': 'test1',
                                'service': { 'resource': 'job/test/',
                                             'type': 'jenkins_testing_service',
                                             'url': 'http://jenkins:8080/',
                                             'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'},
                                'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'}],
               'test_suite_metadata': { '@id': 'test-metadata.json',
                                        'test': [ { 'definition': { 'path': 'test1/sort-and-change-case-test.yml',
                                                                    'test_engine': { 'type': 'planemo',
                                                                                     'version': '>=0.70'}},
                                                    'instance': [ { 'name': 'example_jenkins',
                                                          

In [58]:
# Get the first test suite
response = s.get(f"{lm_base_url}/suites/{suites['items'][0]['uuid']}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
suite = response.json()
utils.pp(suite)

{ 'instances': [ { 'name': 'test1',
                   'service': { 'resource': 'job/test/',
                                'type': 'jenkins_testing_service',
                                'url': 'http://jenkins:8080/',
                                'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'},
                   'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'}],
  'test_suite_metadata': { '@id': 'test-metadata.json',
                           'test': [ { 'definition': { 'path': 'test1/sort-and-change-case-test.yml',
                                                       'test_engine': { 'type': 'planemo',
                                                                        'version': '>=0.70'}},
                                       'instance': [ { 'name': 'example_jenkins',
                                                       'service': { 'resource': 'job/test/',
                                                                    'type': 'jenkins',
                     

In [59]:
# Get the suite status
response = s.get(f"{lm_base_url}/suites/{suites['items'][0]['uuid']}/status")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
suite_status = response.json()
utils.pp(suite_status)

{ 'latest_builds': [ { 'build_id': '4',
                       'instance': { 'name': 'test1',
                                     'service': { 'resource': 'job/test/',
                                                  'type': 'jenkins_testing_service',
                                                  'url': 'http://jenkins:8080/',
                                                  'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'},
                                     'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'},
                       'last_logs': 'Started by user Admin Jenkins\n'
                                    'Running as SYSTEM\n'
                                    'Building in workspace '
                                    '/var/jenkins_home/workspace/test\n'
                                    '[test] $ /bin/sh -xe '
                                    '/tmp/jenkins2038142812728915165.sh\n'
                                    '+ ls -larth\n'
                               

In [60]:
# Get the suite instances
response = s.get(f"{lm_base_url}/suites/{suites['items'][0]['uuid']}/instances")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
instances = response.json()
utils.pp(instances)

{ 'items': [ { 'name': 'test1',
               'service': { 'resource': 'job/test/',
                            'type': 'jenkins_testing_service',
                            'url': 'http://jenkins:8080/',
                            'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'},
               'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'}]}


In [61]:
# Get the first test suite instance
response = s.get(f"{lm_base_url}/instances/{instances['items'][0]['uuid']}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
instance = response.json()
utils.pp(instance)

{ 'name': 'test1',
  'service': { 'resource': 'job/test/',
               'type': 'jenkins_testing_service',
               'url': 'http://jenkins:8080/',
               'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'},
  'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'}


In [62]:
# Get latest builds of the first test suite instance
response = s.get(f"{lm_base_url}/instances/{instance['uuid']}/latest-builds")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
latest_builds = response.json()
utils.pp(latest_builds)

{ 'items': [ { 'build_id': '4',
               'instance': { 'name': 'test1',
                             'service': { 'resource': 'job/test/',
                                          'type': 'jenkins_testing_service',
                                          'url': 'http://jenkins:8080/',
                                          'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'},
                             'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'},
               'last_logs': 'Started by user Admin Jenkins\n'
                            'Running as SYSTEM\n'
                            'Building in workspace '
                            '/var/jenkins_home/workspace/test\n'
                            '[test] $ /bin/sh -xe '
                            '/tmp/jenkins2038142812728915165.sh\n'
                            '+ ls -larth\n'
                            'total 8.0K\n'
                            'drwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 '
                      

In [63]:
# Get the test build '1'
response = s.get(f"{lm_base_url}/instances/{instance['uuid']}/builds/1")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
build1 = response.json()
utils.pp(build1)

{ 'build_id': '1',
  'instance': { 'name': 'test1',
                'service': { 'resource': 'job/test/',
                             'type': 'jenkins_testing_service',
                             'url': 'http://jenkins:8080/',
                             'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'},
                'uuid': 'ec761adf-a41e-4f9b-8421-7da66e085d17'},
  'last_logs': 'Started by user Admin Jenkins\n'
               'Running as SYSTEM\n'
               'Building in workspace /var/jenkins_home/workspace/test\n'
               '[test] $ /bin/sh -xe /tmp/jenkins5791607255757933039.sh\n'
               '+ ls -larth\n'
               'total 8.0K\n'
               'drwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..\n'
               'drwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .\n'
               'Finished: SUCCESS\n',
  'status': 'passed',
  'suite_uuid': '7b2d82c6-eb09-41f6-94d4-5b192cb86c22',
  'timestamp': '1601997377980'}


In [64]:
# Get logs of the test build '1'
response = s.get(f"{lm_base_url}/instances/{instance['uuid']}/builds/1/logs")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
build1 = response.json()
utils.pp(build1)

('Started by user Admin Jenkins\n'
 'Running as SYSTEM\n'
 'Building in workspace /var/jenkins_home/workspace/test\n'
 '[test] $ /bin/sh -xe /tmp/jenkins5791607255757933039.sh\n'
 '+ ls -larth\n'
 'total 8.0K\n'
 'drwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..\n'
 'drwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .\n'
 'Finished: SUCCESS\n')
